In [1]:
# !pip install xmovie
# !conda install -c conda-forge -y ffmpeg

In [21]:
import xesmf as xe
import xarray as xr
import cartopy.crs as ccrs
import numpy as np
import matplotlib.pyplot as plt
import cmocean.cm as cm
from netCDF4 import Dataset
import urllib.request
from IPython.display import Image, display
import gc 
from IPython.display import Video
from IPython.display import HTML

In [3]:
import warnings
from shapely.errors import ShapelyDeprecationWarning
from xmovie import Movie
warnings.filterwarnings(
    action='ignore',
    category=ShapelyDeprecationWarning,  # in cartopy
)
warnings.filterwarnings(
    action="ignore",
    category=UserWarning,
    message=r"No `(vmin|vmax)` provided. Data limits are calculated from input. Depending on the input this can take long. Pass `\1` to avoid this step"
)



## Loading in data

In [4]:
era5_data_path = "/vortexfs1/share/cmip6/data/era5/reanalysis/single-levels/monthly-means/"

#10m u-wind
ds = xr.open_mfdataset(era5_data_path + "surface_latent_heat_flux/*.nc", 
                         engine = "netcdf4")

data = ds["slhf"].sel(longitude = slice(275, 300), latitude = slice(50, 25))
data = data.isel(time = slice(0, 36)).compute() #preload the data, but can also be done without compute

## Making a quick and dirty movie

``xmovie`` allows us to quickly create ``mp4`` or ``gif`` animations in two lines of code!

In [24]:
#save movie
mov = Movie(data, dpi = 200)
mov.save('movie1.mp4', progress=True, 
         overwrite_existing = True,parallel=False, framerate=10)


  0%|          | 0/36 [00:00<?, ?it/s]

Movie created at movie1.mp4


In [25]:
#play movie using HTML
HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="movie1.mp4" type="video/mp4">
</video></div>""")

## Making a presentable movie

The previous animation can be improved by using ``xmovie``'s ``plotfunc`` argument. Using ``plotfunc``, we can specify how we want our plot to look and even animate multiple subfigures at once!

In [26]:

def plotting_function(ds, fig, tt, *args, **kwargs):
    # Define station location for timeseries
        
    ax1 = fig.subplots(ncols=1, subplot_kw={'projection':ccrs.PlateCarree()})
    ax1.coastlines()

    cb = ax1.pcolormesh(ds.longitude, ds.latitude, ds.isel(time=tt), 
                   vmin=-4e7, vmax = 0, 
                    cmap='Reds_r', transform = ccrs.PlateCarree())

    gl = ax1.gridlines(draw_labels = True)
    ax1.set_title(ds.isel(time=tt).time.values)
    ax1.set_aspect(1)
    gl.top_labels = False
    gl.right_labels = False

    fig.colorbar(cb, ax = ax1, label = "Surface Latent Heat Flux\n[J per m^2]")

    return None, None
    # ^ This is not strictly necessary, but otherwise a warning will be raised.

#save movie with cartopy and in parallel 
mov = Movie(data, 
            plotfunc = plotting_function, 
            dpi = 200)

mov.save('movie2.mp4', progress=True, 
         overwrite_existing = True,parallel=False, framerate=10)

  0%|          | 0/36 [00:00<?, ?it/s]

Movie created at movie2.mp4


In [27]:
HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="movie2.mp4" type="video/mp4">
</video></div>""")

## Making an even MORE presentable movie
Okay, so far things look great! Execept... the animations are a little choppy. This is because our data is in monthly resolution. Let's put it onto a weekly resolution and to improve the appearance of our animation. Notice that it takes longer to create this animation since we've increased the amount of data we are using

In [28]:
%%time
# Interpolate time for smoother animation
data_interp = data.resample(time="1W").interpolate("linear")

mov = Movie(data_interp, 
            plotfunc = plotting_function, 
            dpi = 200)

mov.save('movie3.mp4', progress=True, 
         overwrite_existing = True,parallel=False, framerate=10)

  0%|          | 0/153 [00:00<?, ?it/s]

Movie created at movie3.mp4
CPU times: user 45 s, sys: 205 ms, total: 45.2 s
Wall time: 53.4 s


In [29]:
HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="movie3.mp4" type="video/mp4">
</video></div>""")

## Making a presentable animation in parallel
Weekly resolution definitely improves the smoothness of our animation. However, by increasing to 
weekly resolution, we've actually quadrupled our dataset size! One minute doesn't seem like a long time for 3 years worth of animation (it isn't) but we can do better. We can execute animations in parallel. Here, we use 4 workers which should cut our animation time to 25 seconds (it does!). 

In [30]:
%%time

# Interpolate time for smoother animation
data_interp = data.resample(time="1W").interpolate("linear").chunk({"time":1})

mov = Movie(data_interp, 
            plotfunc = plotting_function, 
            dpi = 200)

mov.save('movie4.mp4', progress=True, 
         overwrite_existing = True,parallel=True, framerate=10,
        parallel_compute_kwargs=dict(scheduler="processes", num_workers=4)
)

Movie created at movie4.mp4
CPU times: user 3.03 s, sys: 1.01 s, total: 4.04 s
Wall time: 25.9 s


In [31]:
HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="movie4.mp4" type="video/mp4">
</video></div>""")